In [1]:
import cv2 
import os 
import glob
import json
import csv
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

from tqdm import tqdm 
from module.utils import imread, imfindcircles, findProjectiveTransform, str2array
from module.disp_measure import displacement_measure, homography_transformation

# Import circle detection and APCA parameters 
with open('params.json') as param_config_json : 
    param_config = json.load(param_config_json)



In [ ]:
img_folder = r'\\sss.uos.ac.kr\Volume_2\NIA_Smart_Monitoring\data'
img_list = glob.glob(os.path.join(img_folder, '*.jpg'))
img_list

In [2]:
'''
1. Check if any images where circles detection is not performed.
& Run imfindcircles for recently added images and save to csv 
'''

analysis_info = pd.read_csv('analysis_info.csv')

img_folder = 'imgs'
img_list = glob.glob(os.path.join(img_folder, '*.jpg'))

In [3]:
for img_file in tqdm(img_list) : 
    img_name = os.path.basename(img_file)
    num_image_in_the_cvs = len(np.where(analysis_info['image_name']== img_name)[0])
    num_circles = 0
    
    if num_image_in_the_cvs :  
        img_idx = int(np.where(analysis_info['image_name']== img_name)[0]) 
        num_circles = analysis_info['num_centers'][img_idx] 
    
    if num_image_in_the_cvs == 0 or num_circles != 4 :
        sensor_num = str(os.path.basename(img_name)[4:7])
        if sensor_num in param_config: 
            params = param_config[sensor_num]
            img = imread(img_file)
            sensitivity = params['sensitivity']
            centers, r_estimated, metric = imfindcircles(img, 
                                                 [params['min_rad'], params['max_rad']],
                                                sensitivity = sensitivity)
            circles = np.concatenate((centers, r_estimated[:,np.newaxis]), axis = 0).T
            circles = np.squeeze(circles)
            
        if is_image_in_the_cvs : 
            analysis_info['centers_of_detected_circles'] = circles
            
        else : 
            new_row = [img_name , circles, sensitivity, float('NaN'),  float('NaN'), len(circles)]
            s = pd.Series(new_row, index=analysis_info.columns )
            analysis_info = analysis_info.append(s, ignore_index=True)
            
analysis_info.sort_values(by = 'image_name')
analysis_info.to_csv('analysis_info.csv',index=False)



100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 550.05it/s]


In [4]:
'''
2. Convert circle center coord to displacement 
'''
## This code assumes that displace measurement has been done yet. 
## Need to edit this code to work when previous displacement measurement results exist.
analysis_info = pd.read_csv('analysis_info.csv')

x_displacement_list = []
y_displacement_list = []

# since displacement measurement doesn't take so long, 
# This code will be written just to re-write displacement column everytime 

for num, img_name in enumerate(analysis_info['image_name']) :
    try :
        sric_idx = int(np.where(analysis_info['image_name']== param_config[img_name[4:7]]['src_img'])[0]) 
        src_circles = str2array(analysis_info['centers_of_detected_circles'][sric_idx])
        dest_circles = str2array(analysis_info['centers_of_detected_circles'][num])

        displacement = homography_transformation(src_circles, dest_circles)
        x_displacement_list.append(displacement[0][0])
        y_displacement_list.append(displacement[1][0])
        
    except IndexError as error:
    # Output expected IndexErrors.
        print(img_name, error)
        x_displacement_list.append([0])
        y_displacement_list.append([0])
    except Exception as exception:
        # Output unexpected Exceptions.
        print(img_name, exception)
        x_displacement_list.append([0])
        y_displacement_list.append([0])
    
columns_to_overwrite = ['x_displacement', 'y_displacement']
analysis_info.drop(labels=columns_to_overwrite, axis="columns", inplace=True)


analysis_info['x_displacement'] = x_displacement_list
analysis_info['y_displacement'] = y_displacement_list

analysis_info.to_csv('analysis_info.csv',index=False)
    


Img_147_20200902_180100.jpg index 3 is out of bounds for axis 0 with size 3
Img_148_20200907_190100.jpg index 3 is out of bounds for axis 0 with size 3
Img_148_20200908_010100.jpg index 3 is out of bounds for axis 0 with size 3
Img_168_20200902_130100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200902_140100.jpg index 3 is out of bounds for axis 0 with size 3
Img_168_20200902_160100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200902_220100.jpg index 3 is out of bounds for axis 0 with size 3
Img_168_20200902_230100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200903_010100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200903_020100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200903_030100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200903_050100.jpg index 2 is out of bounds for axis 0 with size 2
Img_168_20200903_060100.jpg index 3 is out of bounds for axis 0 with size 3
Img_168_2020

Img_204_20200827_080100.jpg index 2 is out of bounds for axis 0 with size 2
Img_204_20200829_120100.jpg index 2 is out of bounds for axis 0 with size 2
Img_204_20200829_130100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200829_150101.jpg index 2 is out of bounds for axis 0 with size 2
Img_204_20200829_200100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200829_220100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200829_230100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200830_210100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200902_220100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200902_230100.jpg index 3 is out of bounds for axis 0 with size 3
Img_204_20200903_000100.jpg index 2 is out of bounds for axis 0 with size 2
Img_204_20200903_020100.jpg index 2 is out of bounds for axis 0 with size 2
Img_204_20200903_030100.jpg invalid index to scalar variable.
Img_204_20200903_040100.jp

In [5]:
analysis_info[(sensor_info['APCA_Group'] == 34) ]

,image_name,centers_of_detected_circles,sensitivity,num_centers,x_displacement,y_displacement
0,Img_093_20200806_180100.jpg,[[ 816.66666667 373.88888889 74.83314774]\n...,0.99,4,-1.52963,-4.1487
1,Img_093_20200806_190100.jpg,[[ 820.94594595 565.02702703 74.83314774]\n...,0.99,4,-1.48924,-3.97418
2,Img_093_20200828_180100.jpg,[[ 820.47272727 596.64242424 74.83314774]\n...,0.99,4,-0.927499,-0.487012
3,Img_093_20200828_190100.jpg,[[ 821.90853659 597.00609756 74.83314774]\n...,0.99,4,-0.736312,-0.395371
4,Img_093_20200828_200100.jpg,[[ 822.125 598.05625 74.83314774]\n...,0.99,4,-0.691197,-0.301627
...,...,...,...,...,...,...
9521,Img_208_20200904_100100.jpg,[[1100.37037037 434.12345679 79.84359711]\n...,0.99,4,9.20701,-0.048246
9522,Img_208_20200904_110100.jpg,[[1091.1686747 434.40963855 79.84359711]\n...,0.99,4,7.75267,-0.0661181
9523,Img_208_20200904_120100.jpg,[[1080. 434.32911392 79.84359711]\n...,0.99,4,6.7028,0.00400949
9524,Img_208_20200904_130100.jpg,[[1074.05952381 640.11904762 79.84359711]\n...,0.99,4,5.47318,-0.289679


In [147]:
# to do 

# 1. select apca group 
# 2. get temperature data in op['t']
    # read temperature data according to the excel file 
# 3. get displacement data in op['f']
    # adjust displacement direction using sensor excel file 
# 4. run apca 
# 5. Plot the result 

sensor_info = pd.read_csv('sensor_info.csv')
sensor_info[(sensor_info['APCA_Group'] == 34) ]


,Sensor ID,Sensor Type,Pier,Slab No,APCA_Group,Sensor Direction,Sensor Location,Displacement Direction
83,52,sound,P34,34-S,34,E,S,Forward
84,55,sound,P34,34-S,34,E,S,Forward
85,58,sound,P34,34-S,34,E,S,Forward
88,59,sound,P34,34-S,34,E,S,Forward
89,54,Tmp,P34,34-S,34,E,S,Forward
90,57,Tmp,P34,34-S,34,E,S,Forward
91,60,Tmp,P34,34-S,34,E,S,Forward
92,179,Img,P34,34-S,34,S,S,FALSE
93,180,Img,P34,34-S,34,E,S,Forward
94,181,Tmp,P34,34-S,34,E,S,Forward


In [28]:
'''
3. Check if newly added tmp data and 
Import tmp data and save to csv
'''
# Not sure but, It seems that there are missing temp data. 
analysis_info = pd.read_csv('analysis_info.csv')

tmp_list = []
tmp_file_folder = '/media/sss/Seagate Backup Plus Drive/data/' 


for num, img_name in enumerate(analysis_info['image_name']) :
#     tmp_file_name = glob.glob(os.path.join(tmp_file_folder, '*' + img_name[4:24] + '*.tpr'))
#     tmp_file_name = [s for s in tmp_file_list if img_name[4:24] in s]
#     print('Tmp_' + img_name[4:19] + '0000.tpr' )
    try : 
        f = open(os.path.join(tmp_file_folder, 'Tmp_' + img_name[4:19] + '0000.tpr' ), "r")
        print(f.read())
        
    except : 
        print(25)
        print(os.path.join(tmp_file_folder, 'Tmp_' + img_name[4:19] + '0000.tpr' ))

        
# analysis_info = pd.concat([center_list, pd.DataFrame(x_displacement_list), pd.DataFrame(y_displacement_list)], axis=1)
# analysis_info.to_csv('analysis_info.csv',index=False)

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_01_center.j0000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200806_180000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200806_190000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_180000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_190000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_200000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_210000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_220000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200828_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200829_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200829_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200829_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_093_20200829_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_09

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200829_210000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200829_220000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200829_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_103_20200830_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_10

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_120000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_130000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_140000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_150000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_160000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_170000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_180000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_190000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_111_20200901_200000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_11

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_120000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_130000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_140000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_150000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_160000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_170000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200904_180000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200907_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200907_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_123_20200907_120000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_12

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200831_220000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200831_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_135_20200901_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_13

...

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200907_220000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200907_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_164_20200908_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_16

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_120000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_130000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_140000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_175_20200829_150000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_17

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200902_220000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200902_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_187_20200903_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_18

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200829_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_196_20200830_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_19

25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200902_230000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_000000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_010000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_020000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_030000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_040000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_050000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_060000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_070000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_080000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_090000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_100000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_204_20200903_110000.tpr
25
/media/sss/Seagate Backup Plus Drive/data/Tmp_20

In [6]:
path = r'C:\Users\shdik\OneDrive\바탕 화면'
disp_pd = pd.read_csv(os.path.join(path, 'analysis_info.csv'))
temp_ps = pd.read_csv(os.path.join(path, 'imgTempr_list_20200828_20200908 (1).csv'))


date_list = []

for img_name in disp_pd['image_name']:
    if '_093_' in img_name :
        date_list.append(img_name[8:-4])

sensor_list = []

for img_name in disp_pd['image_name']:
    sensor_list.append(img_name[4:7])
    
sensor_uniq_list = np.unique(sensor_list)


for idx in range(len(sensor_uniq_list)) :

    sensor = sensor_uniq_list[idx]
    disp_list = []
    tmp_list = []
    for row in disp_pd.iterrows() :
        if sensor in row[1][0] : 
            disp_list.append(row[1][4])
    #         print()
            temp_row_index = temp_ps.loc[temp_ps['파일명'] == row[1][0]]

            for value in temp_row_index['온도값(℃)']:
                tmp_list.append(value)

    fig, ax1 = plt.subplots(figsize=(20, 20))
    

    color = 'tab:red'
    ax1.set_xlabel('time', fontsize = 20.0)
    ax1.set_ylabel('Temperaure(Celcius)', color=color, fontsize = 20.0)
    ax1.plot(tmp_list, color=color)
    ax1.tick_params(axis='y', labelcolor=color, labelsize = 24)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('Displacement', color=color, fontsize = 20.0)  # we already handled the x-label with ax1
    ax2.plot(disp_list, color=color)
    ax2.tick_params(axis='y', labelcolor=color, labelsize = 24)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.title('Sensor No.' + str(sensor),  size=20)
#     plt.show()
    plt.savefig(os.path.join(path, 'Sensor No.' + str(sensor) + '_y.png'  ))
    plt.close




20200821100000425, 26.5
20200821101000882, 26.6
20200821102000466, 26.7
20200821103000079, 26.8
20200821104000636, 26.9
20200821105000848, 27.0



In [ ]:
for v, k in param_config.items():

    param_config[v]['src_img'] = 'Img_' + v + '_20200901_160100.jpg'
    
with open('params.json', 'w', encoding='utf-8') as f:
    json.dump(param_config, f)
#     try:
#         del param_config[v]['src_center'] 
#     except : 
#         print('src_center does not exist')